In [1]:
import asyncio
import awswrangler as wr
from datetime import datetime, timedelta
from IPython.display import display
from metaapi_cloud_sdk import MetaApi
import os
import pandas as pd
import pytz
import sys
from time import sleep

In [2]:
repo_name = "mercury-pipeline-metaapi-sam"
sys.path.append(os.getcwd().split(repo_name)[0] + repo_name + "/lambda") 

In [3]:
from src.config import *
from src.io import *
from src.utils import *
from src.access import get_demo_account

In [4]:
from src.ticks.clean.get import get_clean_ticks
from src.candles.from_ticks.get import get_candles_from_ticks
from src.candles.transform.save import ffill

In [5]:
async def yield_raw_ticks_between_datetimes(
    account: object,
    symbol: str,
    datetimeutc_from: datetime,
    datetimeutc_to: datetime,
    pause: int = 5
):
    offset = 0
    while True:
        l_ticks_i = await account.get_historical_ticks(
            symbol,
            start_time=datetimeutc_from,
            limit=1000,
            offset=offset,
        )
        if len(l_ticks_i) == 0:
            break
        yield offset, l_ticks_i
        await asyncio.sleep(pause)
        datetimeutc_last = l_ticks_i[-1]["time"]
        offset += len(l_ticks_i)
        if datetimeutc_last >= datetimeutc_to:
            break

In [6]:
symbol = "GBPUSD"
frequency = "15T"

In [7]:
datetimestr_from = (
    pytz
    .timezone("Europe/London")
    .localize(datetime.now())
    .astimezone(pytz.timezone("UTC"))
    .strftime("%Y%m%d000000000000")
)

datetimestr_to = (
    pytz
    .timezone("Europe/London")
    .localize(datetime.now() + timedelta(days=1))
    .astimezone(pytz.timezone("UTC"))
    .strftime("%Y%m%d%H%M%S%f")
)
datetimestr_from, datetimestr_to

('20220708000000000000', '20220709161104015631')

In [8]:
# Get demo account
account = await get_demo_account()

# Convert datetimes from string to datetime
datetimeutc_from = pytz.timezone("UTC").localize(datetime.strptime(datetimestr_from, "%Y%m%d%H%M%S%f"))
datetimeutc_to = pytz.timezone("UTC").localize(datetime.strptime(datetimestr_to, "%Y%m%d%H%M%S%f"))

In [9]:
generator_async = yield_raw_ticks_between_datetimes(
    account=account,
    symbol=symbol,
    datetimeutc_from=datetimeutc_from,
    datetimeutc_to=datetimeutc_to,
)

In [10]:
# Before I do anything, I need to get the last candle from the previous night
# from clean/candles/from_ticks






i = 0
df_ticks_clean_prev = pd.DataFrame()
async for offset, ticks_raw in generator_async:
    
    print(offset)
    
    if ticks_raw == []:
        sleep(10)
        continue

    df_ticks_clean = get_clean_ticks(ticks_raw)
    df_ticks_clean = pd.concat([
        df_ticks_clean_prev,
        df_ticks_clean
    ])
    
    df_candles_from_ticks = get_candles_from_ticks(
        df_ticks_clean,
        frequency
    )

    df_candles_ffill = ffill(df_candles_from_ticks)
    
    s_candle_last = df_candles_ffill.iloc[-1]
    df_candles_out = df_candles_ffill.iloc[:-1].copy() # could be empty so beware
    
    mask_last = df_ticks_clean["timestamp_utc"] >= s_candle_last["timestamp_utc.open"]
    df_ticks_clean_prev = df_ticks_clean[mask_last].copy()
    
    display(df_candles_out)

    i += 1
    if i >= 20:
        break
    sleep(1)

0


,timestamp_utc.open,ask.open,ask.close,ask.low,ask.high,n_ticks,bid.open,bid.close,bid.low,bid.high,ask.idx_hi_lo,bid.idx_hi_lo,timestamp_utc.close,timestamp_uk.open,timestamp_uk.close


1000


,timestamp_utc.open,ask.open,ask.close,ask.low,ask.high,n_ticks,bid.open,bid.close,bid.low,bid.high,ask.idx_hi_lo,bid.idx_hi_lo,timestamp_utc.close,timestamp_uk.open,timestamp_uk.close


2000


...

,timestamp_utc.open,ask.open,ask.close,ask.low,ask.high,n_ticks,bid.open,bid.close,bid.low,bid.high,ask.idx_hi_lo,bid.idx_hi_lo,timestamp_utc.close,timestamp_uk.open,timestamp_uk.close
0,2022-07-08 00:30:00,1.203,1.20329,1.20272,1.20343,478,1.20287,1.20317,1.20259,1.20331,lo,lo,2022-07-08 00:45:00,2022-07-08 01:30:00,2022-07-08 01:45:00


18000


,timestamp_utc.open,ask.open,ask.close,ask.low,ask.high,n_ticks,bid.open,bid.close,bid.low,bid.high,ask.idx_hi_lo,bid.idx_hi_lo,timestamp_utc.close,timestamp_uk.open,timestamp_uk.close


19000


,timestamp_utc.open,ask.open,ask.close,ask.low,ask.high,n_ticks,bid.open,bid.close,bid.low,bid.high,ask.idx_hi_lo,bid.idx_hi_lo,timestamp_utc.close,timestamp_uk.open,timestamp_uk.close


In [11]:
assert False

AssertionError: 

In [ ]:
offset, ticks_raw = await generator_async.__anext__()

In [ ]:
ticks_raw[0], ticks_raw[-1]

In [ ]:
df_ticks_clean = get_clean_ticks(ticks_raw)
df_ticks_clean

In [ ]:
df_candles_from_ticks = get_candles_from_ticks(
    df_ticks_clean,
    frequency
)
df_candles_from_ticks

In [ ]:
df_candles_ffill = ffill(df_candles_from_ticks)
df_candles_ffill

In [ ]:
s_candle_last = df_candles_ffill.iloc[-1]
s_candle_last

In [ ]:
df_candles_out = df_candles_ffill.iloc[:-1].copy()
df_candles_out

In [ ]:
df_ticks_clean_last = df_ticks_clean[df_ticks_clean["timestamp_utc"] >= s_candle_last["timestamp_utc.open"]].copy()
df_ticks_clean_last

In [ ]:
offset, ticks_raw = await generator_async.__anext__()

In [ ]:
df_ticks_clean = get_clean_ticks(ticks_raw)
df_ticks_clean

In [ ]:
df_ticks_clean = pd.concat([
    df_ticks_clean_last,
    df_ticks_clean
])
df_ticks_clean

In [ ]:
df_candles_from_ticks = get_candles_from_ticks(
    df_ticks_clean,
    frequency
)
df_candles_from_ticks

In [ ]:
df_candles_ffill = ffill(df_candles_from_ticks)
df_candles_ffill

In [ ]:
s_candle_last = df_candles_ffill.iloc[-1]
s_candle_last

In [ ]:
df_candles_out = df_candles_ffill.iloc[:-1].copy()
df_candles_out

In [ ]:
df_ticks_clean_last = df_ticks_clean[df_ticks_clean["timestamp_utc"] >= s_candle_last["timestamp_utc.open"]].copy()
df_ticks_clean_last